## Getting embedding for each column value using Chronos 
- Saving this as npz array due to time constraint on jupyter notebook

In [1]:
import pandas as pd
import torch
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

df = pd.read_parquet('processed-data/processed-set-a.parquet').drop(columns=["ICUType"])
df= df.fillna(0)

print(list(df.columns))


['Time', 'Age', 'BUN', 'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'HR', 'Height', 'K', 'MAP', 'MechVent', 'Mg', 'NIDiasABP', 'NIMAP', 'NISysABP', 'Na', 'PaCO2', 'PaO2', 'Platelets', 'RecordID', 'SaO2', 'SysABP', 'Temp', 'Urine', 'WBC', 'Weight', 'pH', 'ALP', 'ALT', 'AST', 'Albumin', 'Bilirubin', 'Cholesterol', 'Lactate', 'RespRate', 'TroponinI', 'TroponinT']


In [13]:
df.head()

,Time,Age,BUN,Creatinine,DiasABP,GCS,Gender,Glucose,HCO3,HCT,...,Bilirubin,Cholesterol,FiO2,MechVent,NIDiasABP,NIMAP,NISysABP,SaO2,TroponinI,TroponinT
0,00:00,72.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01:00,72.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02:00,72.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,03:00,72.0,0.0,0.0,58.5,13.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,04:00,72.0,9.0,0.9,57.0,14.0,1.0,129.0,27.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
import numpy as np
unique_values = list(df["RecordID"].unique())
count_record_id = 0
# averaged_embeddings =[]
# all_record_embeddings =[]
all_full_record_embeddings =[]
for record_id in unique_values:
    embeddings_of_all_columns =[]
    full_embeddings_of_all_columns = []
    for column in list(df.columns):
        if column == "Time":
            continue
        context = df[df["RecordID"] == record_id][column]
        context = torch.tensor(context.to_numpy()) #size 49 - from time 00:00 to 48:00

        embeddings, tokenizer_state = pipeline.embed(context)
        embeddings = embeddings.to(torch.float32).cpu().numpy() #shape [1,50, 512]
        
        # arr_avg = embeddings.mean(axis=1)  #Average over the token dimension -> shape [1, 512]
        # arr_squeezed = np.squeeze(arr_avg, axis=0) #Squeeze to shape (512,)
        #print(arr_squeezed.shape) 
        # embeddings_of_all_columns.append(arr_squeezed)
        full_embeddings_of_all_columns.append(embeddings.squeeze(axis=0))
        
    #stacked_for_one_record = np.stack(embeddings_of_all_columns, axis=0)
    stacked_full_for_one_record = np.stack(full_embeddings_of_all_columns, axis=0)
    print(stacked_full_for_one_record.shape)
    # all_record_embeddings.append(stacked_for_one_record)
    string = f"chronos_full_emb_train/{count_record_id}.npz"
    np.savez(string, stacked_full_for_one_record)
    #all_full_record_embeddings.append(stacked_full_for_one_record)
    #print("stack_2d.shape",stack_2d.shape) #(41, 512)
    # Average across columns -> shape [500]
    # avg_over_columns = stack_2d.mean(axis=0)
    # averaged_embeddings.append(avg_over_columns)
    print("count_record_id",count_record_id)
    count_record_id+=1     
# final_embeddings = np.stack(all_full_record_embeddings, axis=0)
# print(final_embeddings.shape)
# string = f"chronos_final_emb_train.npz"
# np.savez(string, final_embeddings)
# string = f"chronos_embeddings_avg_train.npz"
# np.savez(string, averaged_embeddings)

(41, 50, 512)
count_record_id 0
(41, 50, 512)
count_record_id 1
(41, 50, 512)
count_record_id 2
(41, 50, 512)
count_record_id 3
(41, 50, 512)
count_record_id 4
(41, 50, 512)
count_record_id 5
(41, 50, 512)
count_record_id 6
(41, 50, 512)
count_record_id 7
(41, 50, 512)
count_record_id 8
(41, 50, 512)
count_record_id 9
(41, 50, 512)
count_record_id 10
(41, 50, 512)
count_record_id 11
(41, 50, 512)
count_record_id 12
(41, 50, 512)
count_record_id 13
(41, 50, 512)
count_record_id 14
(41, 50, 512)
count_record_id 15
(41, 50, 512)
count_record_id 16
(41, 50, 512)
count_record_id 17
(41, 50, 512)
count_record_id 18
(41, 50, 512)
count_record_id 19
(41, 50, 512)
count_record_id 20
(41, 50, 512)
count_record_id 21
(41, 50, 512)
count_record_id 22
(41, 50, 512)
count_record_id 23
(41, 50, 512)
count_record_id 24
(41, 50, 512)
count_record_id 25
(41, 50, 512)
count_record_id 26
(41, 50, 512)
count_record_id 27
(41, 50, 512)
count_record_id 28
(41, 50, 512)
count_record_id 29
(41, 50, 512)
count_

## test Set 

In [2]:
import pandas as pd
import torch
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

df = pd.read_parquet('processed-data/processed-set-c.parquet').drop(columns=["ICUType"])
df= df.fillna(0)

print(list(df.columns))

import numpy as np
unique_values = list(df["RecordID"].unique())
count_record_id = 0
# averaged_embeddings =[]
# all_record_embeddings =[]
all_full_record_embeddings_val =[]
for record_id in unique_values:
    embeddings_of_all_columns =[]
    full_embeddings_of_all_columns = []
    for column in list(df.columns):
        if column == "Time":
            continue
        context = df[df["RecordID"] == record_id][column]
        context = torch.tensor(context.to_numpy()) #size 49 - from time 00:00 to 48:00

        embeddings, tokenizer_state = pipeline.embed(context)
        embeddings = embeddings.to(torch.float32).cpu().numpy() #shape [1,50, 512]
        
        # arr_avg = embeddings.mean(axis=1)  #Average over the token dimension -> shape [1, 512]
        # arr_squeezed = np.squeeze(arr_avg, axis=0) #Squeeze to shape (512,)
        #print(arr_squeezed.shape) 
        # embeddings_of_all_columns.append(arr_squeezed)
        full_embeddings_of_all_columns.append(embeddings.squeeze(axis=0))
        
    #stacked_for_one_record = np.stack(embeddings_of_all_columns, axis=0)
    stacked_full_for_one_record = np.stack(full_embeddings_of_all_columns, axis=0)
    print(stacked_full_for_one_record.shape)
    # all_record_embeddings.append(stacked_for_one_record)
    #string = f"chronos_full_emb_test/{count_record_id}.npz"
    # np.savez(string, stacked_full_for_one_record)
    
    # all_record_embeddings.append(stacked_for_one_record)
    string = f"chronos_full_emb_test/{count_record_id}.npz"
    np.savez(string, stacked_full_for_one_record)
    
    #all_full_record_embeddings.append(stacked_full_for_one_record)
    #print("stack_2d.shape",stack_2d.shape) #(41, 512)
    # Average across columns -> shape [500]
    # avg_over_columns = stack_2d.mean(axis=0)
    # averaged_embeddings.append(avg_over_columns)
    print("count_record_id",count_record_id)
    count_record_id+=1     

# print(final_embeddings.shape)
# string = f"chronos_final_emb_train.npz"
# np.savez(string, final_embeddings)
# string = f"chronos_embeddings_avg_train.npz"
# np.savez(string, averaged_embeddings)

['Time', 'Age', 'BUN', 'Creatinine', 'DiasABP', 'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'HR', 'Height', 'K', 'Lactate', 'MAP', 'Mg', 'Na', 'PaCO2', 'PaO2', 'Platelets', 'RecordID', 'RespRate', 'SysABP', 'Temp', 'Urine', 'WBC', 'Weight', 'pH', 'ALP', 'ALT', 'AST', 'Albumin', 'Bilirubin', 'Cholesterol', 'FiO2', 'MechVent', 'NIDiasABP', 'NIMAP', 'NISysABP', 'SaO2', 'TroponinI', 'TroponinT']
(41, 50, 512)
count_record_id 0
(41, 50, 512)
count_record_id 1
(41, 50, 512)
count_record_id 2
(41, 50, 512)
count_record_id 3
(41, 50, 512)
count_record_id 4
(41, 50, 512)
count_record_id 5
(41, 50, 512)
count_record_id 6
(41, 50, 512)
count_record_id 7
(41, 50, 512)
count_record_id 8
(41, 50, 512)
count_record_id 9
(41, 50, 512)
count_record_id 10
(41, 50, 512)
count_record_id 11
(41, 50, 512)
count_record_id 12
(41, 50, 512)
count_record_id 13
(41, 50, 512)
count_record_id 14
(41, 50, 512)
count_record_id 15
(41, 50, 512)
count_record_id 16
(41, 50, 512)
count_record_id 17
(41, 50, 512)
count_re

NameError: name 'all_full_record_embeddings' is not defined

## Linear Probe on aggregated chronous embeddings 

In [1]:
import numpy as np
import glob
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score,roc_auc_score


data = np.load("chronos_embeddings_avg_train.npz")
arr_keys = data.files
if len(arr_keys) == 1:
    X_train = data[arr_keys[0]]
    print(X_train.shape)
else:
    print(f"Multiple arrays in {file_path}: {arr_keys}")

y_train = pd.read_parquet('processed-data/processed-outcomes-a.parquet')["In-hospital_death"].to_numpy().flatten()

data = np.load("chronos_embeddings_avg_val.npz")
arr_keys = data.files
if len(arr_keys) == 1:
    X_val= data[arr_keys[0]]
    
else:
    print(f"Multiple arrays in {file_path}: {arr_keys}")

y_val = pd.read_parquet('processed-data/processed-outcomes-b.parquet')["In-hospital_death"].to_numpy().flatten()

X_train = np.concatenate((X_train, X_val), axis=0)
y_train = np.concatenate((y_train, y_val), axis=0)
print(X_val.shape)

data = np.load("chronos_embeddings_avg_test.npz")

arr_keys = data.files
if len(arr_keys) == 1:
    X_test = data[arr_keys[0]]
    print(X_test.shape)
else:
    print(f"Multiple arrays in {file_path}: {arr_keys}")

y_test = pd.read_parquet('processed-data/processed-outcomes-c.parquet')["In-hospital_death"].to_numpy().flatten()

# Initialize and train the logistic regression
log_reg = LogisticRegression()  # you can specify hyperparams if you wish
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict_proba(X_test)[:, 1] 

roc = roc_auc_score(y_test, y_pred)  # Area under the ROC curve
prc = average_precision_score(y_test, y_pred)  # Area under the Precision-Recall curve
print("ROC AUC:", roc)
print("PR AUC:", prc)

(4000, 512)
(4000, 512)
(4000, 512)
ROC AUC: 0.4968086996783922
PR AUC: 0.15289012798108775


## Training NN 

In [5]:
#X = final_embeddings
import pandas as pd
import torch
import numpy as np
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
from sklearn.metrics import accuracy_score, roc_auc_score
import glob

class ChronosEmbeddingsDataset(Dataset):
    def __init__(self, folder_path, labels):
        """
        Args:
            folder_path (str): Path to folder containing .npz files.
            labels (list or np.array): Labels (float or int), same length as number of .npz files.
        """
        super().__init__()
        self.npz_files = sorted(glob.glob(f"{folder_path}/*.npz"))
        self.labels = labels  # shape [N]
        if len(self.npz_files) != len(self.labels):
            raise ValueError(
                f"Number of NPZ files ({len(self.npz_files)}) != number of labels ({len(labels)})"
            )

    def __len__(self):
        return len(self.npz_files)

    def __getitem__(self, idx):
        # 1) Load the .npz on the fly
        file_path = self.npz_files[idx]
        data = np.load(file_path)
        arr_keys = data.files
        if len(arr_keys) != 1:
            raise RuntimeError(f"Expected exactly one array per file, got {arr_keys}")

        # 2) Convert the array to torch.Tensor
        # shape: [41,50,512]
        x = torch.from_numpy(data[arr_keys[0]]).float()

        # 3) Get the label and convert to tensor
        y = torch.tensor(self.labels[idx], dtype=torch.float32)

        # 4) Return (input, label)
        return x, y

In [ ]:
# Suppose 'y' is a list/array of labels, length N
y = pd.read_parquet('processed-data/processed-outcomes-a.parquet')["In-hospital_death"].to_numpy().flatten()

# Create dataset
full_dataset = ChronosEmbeddingsDataset("chronos_full_emb_train", y)

# 80/20 train/val split
val_ratio = 0.2
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [ ]:
# Create your DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False)

In [8]:

# Model
class PatientTimeSeriesClassifier(nn.Module):
    def __init__(self, embedding_dim=512, num_timesteps=50, num_variables=41, hidden_dim=128):
        super().__init__()
        
        # Temporal encoder: shared across variables
        self.temporal_encoder = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)  # Output shape: (B * 41, hidden_dim, 1)
        )
        
        # Attention across variables
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)  # scalar attention weight per variable
        )

        # Final classifier
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        # x: (B, 41, 50, 512)
        B, V, T, D = x.shape

        # Reshape to (B * V, T, D) → for temporal encoding
        x = x.view(B * V, T, D).transpose(1, 2)  # (B*V, D, T)
        temp_encoded = self.temporal_encoder(x).squeeze(-1)  # (B*V, hidden_dim)

        # Reshape back to (B, V, hidden_dim)
        temp_encoded = temp_encoded.view(B, V, -1)

        # Compute attention weights over variables
        attn_scores = self.attention(temp_encoded)  # (B, V, 1)
        attn_weights = F.softmax(attn_scores, dim=1)  # (B, V, 1)

        # Weighted sum over variables → (B, hidden_dim)
        patient_rep = torch.sum(attn_weights * temp_encoded, dim=1)

        # Classify
        logits = self.classifier(patient_rep).squeeze(-1)  # (B,)
        probs = torch.sigmoid(logits)  # for BCE loss
        return probs, attn_weights.squeeze(-1)  # (B,), (B, V)

In [9]:



# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = PatientTimeSeriesClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()
num_epochs = 50
# Early stopping and checkpointing setup
best_auc = 0.0
epochs_no_improve = 0
patience = 10
checkpoint_path = "best_model.pt"

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.to(device)   # shape: (batch_size, 41, 50, 512)
        labels = labels.to(device)   # shape: (batch_size,)

        optimizer.zero_grad()
        print(inputs.shape)
        probs, _ = model(inputs)
        loss = criterion(probs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_probs, val_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)   # shape: (batch_size, 41, 50, 512)
            labels = labels.to(device)   # shape: (batch_size,)

            probs, _ = model(inputs)
            val_probs.append(probs)
            val_labels.append(labels)
    val_probs = torch.cat(val_probs).cpu().numpy()
    val_labels = torch.cat(val_labels).cpu().numpy()

    val_loss = criterion(torch.tensor(val_probs), torch.tensor(val_labels)).item()
    val_acc = accuracy_score(val_labels, val_probs > 0.5)
    val_auc = roc_auc_score(val_labels, val_probs)

    print(f"Epoch {epoch}: Val Loss = {val_loss:.4f}, Acc = {val_acc:.4f}, AUC = {val_auc:.4f}")

    # Early stopping logic
    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), checkpoint_path)
        epochs_no_improve = 0
        print(f"✅ New best model saved with AUC = {val_auc:.4f}")
    else:
        epochs_no_improve += 1
        print(f"⚠️ No improvement. Patience: {epochs_no_improve}/{patience}")
        if epochs_no_improve >= patience:
            print("⏹️ Early stopping triggered.")
            break

# Load best model after training
model.load_state_dict(torch.load(checkpoint_path))
print("🏁 Loaded best model from checkpoint.")

Using device: cuda
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 50, 512])
torch.Size([32, 41, 5

In [10]:
checkpoint_path = "best_model.pt"
model = PatientTimeSeriesClassifier()
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

y = pd.read_parquet('processed-data/processed-outcomes-c.parquet')["In-hospital_death"].to_numpy().flatten()

# Create dataset
full_dataset = ChronosEmbeddingsDataset("chronos_full_emb_test", y)

test_loader = DataLoader(full_dataset, batch_size=64, shuffle=False)

from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

model.eval()
all_probs = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        probs, _ = model(inputs)                  # shape: (batch_size,)
        all_probs.append(probs)
        all_labels.append(labels)

# Stack all batches together
all_probs = torch.cat(all_probs).numpy()
all_labels = torch.cat(all_labels).numpy()

# Binary predictions
y_pred = (all_probs > 0.5).astype(int)

# Metrics
acc = accuracy_score(all_labels, y_pred)
roc = roc_auc_score(all_labels, all_probs)
prc = average_precision_score(all_labels, all_probs)

print(f"Test Accuracy     = {acc:.4f}")
print(f"Test ROC AUC      = {roc:.4f}")
print(f"Test PR AUC       = {prc:.4f}")


Test Accuracy     = 0.8538
Test ROC AUC      = 0.5047
Test PR AUC       = 0.1480
